In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
today = datetime.today()
# Библиотеки для моделирования
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# Стоп-слова, знаки пунктуации
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words("russian")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oigla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oigla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
plan = pd.read_csv('I:/Download/Датасеты/test_plan_01.csv')
test = pd.read_csv('I:/Download/Датасеты/test.csv')

In [3]:
tfidf = TfidfVectorizer(stop_words=list(stop_words))
plan['type3'] = plan['type3'].fillna('')
overview_matrix = tfidf.fit_transform(plan['type3'])
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
mapping = pd.Series(plan.index,index =plan['type2'])
print(mapping)

type2
ОНЛАЙН Красота и стиль           0
ОНЛАЙН Домоводство               1
ОНЛАЙН Домоводство               2
ОНЛАЙН Домоводство               3
ОНЛАЙН Домоводство               4
                             ...  
Информационные технологии    59008
Иностранные языки            59009
Иностранные языки            59010
Спортивные игры              59011
Информационные технологии    59012
Length: 59013, dtype: int64


In [4]:
plan.index

RangeIndex(start=0, stop=59013, step=1)

In [5]:
overview_matrix

<59013x458 sparse matrix of type '<class 'numpy.float64'>'
	with 178006 stored elements in Compressed Sparse Row format>

In [6]:
similarity_matrix

array([[1.       , 0.0271926, 0.0271926, ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [7]:
def recommend_plan(plan_input):
    plan_index = mapping[plan_input]
    similarity_score = list(enumerate(similarity_matrix[plan_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
    similarity_score = similarity_score[0:10]
    plan_indices = [i[0] for i in similarity_score]
    return (plan['numgroup'].iloc[plan_indices])

In [8]:
print(recommend_plan('ОНЛАЙН Гимнастика'))

8323    801352526
8322    801350520
8321    801349601
8320    801369467
8319    801366648
8318    801373454
8317    801353771
8316    801352950
8315    801351464
8314    801347511
Name: numgroup, dtype: int64
